Machine Learning - Lab 01 - Feature Engineering

Import Libraries

In [7]:
import pandas as pd
import numpy as np
import xgboost as xg
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.svm import SVC
from sklearn import metrics

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif, f_regression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestClassifier

In [8]:
LABELS=['label_1','label_2','label_3','label_4']
FEATURES=[f'feature_{i}' for i in range(1,257)]

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
speaker_data_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/CS4622 - Machine Learning/Lab1/train.csv")
speaker_data_train.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_251,feature_252,feature_253,feature_254,feature_255,feature_256,label_1,label_2,label_3,label_4
0,-1.928826,-0.291540,1.105909,2.070652,0.657838,-0.670940,-0.968238,0.049916,1.111149,-1.834592,...,-0.691538,-1.152522,0.441157,-1.494434,-0.344974,-0.395641,45,NaN,1,6
1,-1.709277,-1.008998,0.956611,4.313823,-0.669455,1.928765,0.162550,0.152173,2.356505,-2.898549,...,-0.668235,-0.532695,0.151163,-1.876885,1.651534,-2.706490,45,NaN,1,6
2,-1.906183,-0.357562,-0.682627,4.651838,-0.884940,1.159512,1.354481,-1.432248,2.160144,-3.175735,...,-0.026728,0.440408,2.745906,-0.680754,-0.643588,-2.587203,45,NaN,1,6
3,0.697872,-2.218567,-0.572214,1.192062,0.329554,0.741364,0.962354,0.993512,2.896854,-1.600380,...,0.709032,0.812106,1.055266,-0.915258,-0.924856,-1.318964,45,NaN,1,6
4,-1.976895,-0.496308,0.368102,3.739787,-0.132058,1.234980,-0.501003,0.881554,1.630941,-3.853953,...,-0.599700,-1.024829,1.869753,-2.498570,3.268253,-1.356606,45,NaN,1,6


In [11]:
speaker_data_valid = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/CS4622 - Machine Learning/Lab1/valid.csv")
speaker_data_valid.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_251,feature_252,feature_253,feature_254,feature_255,feature_256,label_1,label_2,label_3,label_4
0,-1.153148,-1.038098,1.419872,2.734152,1.154604,-1.086937,-0.516225,-1.370325,2.865359,-1.879877,...,-0.719046,-1.248530,0.144460,-3.240056,0.052614,0.083108,45,NaN,1,6
1,-1.377524,-1.018393,1.102352,2.849025,0.440302,-1.149039,-0.789796,-2.258196,1.264268,-2.123730,...,-1.383339,-0.787736,1.044895,-2.289637,0.199752,-0.712154,45,NaN,1,6
2,0.889574,-2.743300,-0.320194,3.047766,-0.923335,1.741686,-0.615148,0.756482,2.074775,-1.433126,...,1.178322,0.035333,0.857712,-1.928684,0.639870,-0.268576,45,NaN,1,6
3,-1.527213,-1.133121,0.385927,3.129767,0.229020,1.373105,0.919284,-0.755558,1.086973,-2.440614,...,-0.664176,0.805059,0.975368,-2.700269,1.523236,-1.259052,45,NaN,1,6
4,0.948176,-0.750248,0.008329,1.675338,1.941155,-0.783623,-0.485584,-0.261882,2.875204,-1.473030,...,-0.665200,-0.428060,-0.393100,-1.854657,2.207063,-0.342725,45,NaN,1,6


Standardizing Features

In [12]:
X_train = {}
X_valid = {}
y_train = {}
y_valid = {}

for label in LABELS:
  if (label == 'label_2'):
    speaker_data_train_df = speaker_data_train.dropna()
    speaker_data_valid_df=speaker_data_valid.dropna()

  else:
    speaker_data_train_df = speaker_data_train
    speaker_data_valid_df = speaker_data_valid

  scaler = StandardScaler()
  X_train[label] = pd.DataFrame(scaler.fit_transform(speaker_data_train_df.drop(LABELS,axis=1)),columns=FEATURES)
  y_train[label] = speaker_data_train_df[label]
  X_valid[label] = pd.DataFrame(scaler.transform(speaker_data_valid_df.drop(LABELS,axis=1)),columns=FEATURES)
  y_valid[label] = speaker_data_valid_df[label]

y_train['label_2']

480      25.0
481      25.0
482      25.0
483      25.0
484      25.0
         ... 
28515    29.0
28516    29.0
28517    29.0
28518    29.0
28519    29.0
Name: label_2, Length: 28040, dtype: float64

Label - 01

Removing Low-Variance Features using Variance Threshold Approach

In [13]:
variance_threshold = VarianceThreshold(threshold=0)
variance_threshold.fit(X_train['label_1'])

X_train['label_1'].columns[variance_threshold.get_support()]

low_variance_columns = []

for column in X_train['label_1'].columns:
    if (column not in X_train['label_1'].columns[variance_threshold.get_support()]):
        low_variance_columns.append(column)

print("Number of constant columns : ", len(low_variance_columns))

Number of constant columns :  0


Finding Pairwise Correlation between Features

In [14]:
def correlation(data, threshold):
    column_correlation = set()  # Set of all the names of correlated columns
    correlation_matrix = data.corr()

    for i in range(len(correlation_matrix.columns)):
        for j in range(i):
            if abs(correlation_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                column_name = correlation_matrix.columns[i]  # getting the name of column
                column_correlation.add(column_name)

    return column_correlation


correlation_features = correlation(X_train['label_1'], 0.85)
print("Number of correlation features : ", len(set(correlation_features)))
print(correlation_features)

Number of correlation features :  0
set()


Calculate the Accuracy before applying any Feature Selection Approaches

In [15]:
classifier = SVC(kernel='linear')
classifier.fit(X_train['label_1'],y_train['label_1'])
y_predict = classifier.predict(X_valid['label_1'])

print("Accuracy : ", metrics.accuracy_score(y_valid['label_1'],y_predict))
print("Precision : ", metrics.precision_score(y_valid['label_1'],y_predict, average='weighted'))
print("Recall : ", metrics.recall_score(y_valid['label_1'],y_predict, average='weighted'))

Accuracy :  0.9906666666666667
Precision :  0.9914608132608133
Recall :  0.9906666666666667


Univariate Feature Selection using SelectKBest Approach

In [16]:
selector = SelectKBest(f_classif, k=150)
X_train_new_values_L1 = selector.fit_transform(X_train['label_1'], y_train['label_1'])
print('Shape after SelectKBest Approach : ', X_train_new_values_L1.shape)

classifier.fit(X_train_new_values_L1,y_train['label_1'])
y_predict = classifier.predict(selector.transform(X_valid['label_1']))

print("Accuracy : ", metrics.accuracy_score(y_valid['label_1'],y_predict))
print("Precision : ", metrics.precision_score(y_valid['label_1'],y_predict, average='weighted'))
print("Recall : ", metrics.recall_score(y_valid['label_1'],y_predict, average='weighted'))

Shape after SelectKBest Approach :  (28520, 150)
Accuracy :  0.9813333333333333
Precision :  0.9826210171894383
Recall :  0.9813333333333333


Dimensionality Reduction using Principal Component Analysis Approach

In [17]:
principal_component_analysis = PCA(n_components=0.97,svd_solver='full')
principal_component_analysis.fit(X_train['label_1'])
X_train_trf = pd.DataFrame(principal_component_analysis.transform(X_train['label_1']))
X_valid_trf = pd.DataFrame(principal_component_analysis.transform(X_valid['label_1']))
print('Shape after Principal Component Analysis : ', X_train_trf.shape)

classifier.fit(X_train_trf,y_train['label_1'])
y_predict=classifier.predict(principal_component_analysis.transform(X_valid['label_1']))

print("Accuracy : ", metrics.accuracy_score(y_valid['label_1'],y_predict))
print("Precision : ", metrics.precision_score(y_valid['label_1'],y_predict, average='weighted'))
print("Recall : ", metrics.recall_score(y_valid['label_1'],y_predict, average='weighted'))

Shape after Principal Component Analysis :  (28520, 80)
Accuracy :  0.9853333333333333
Precision :  0.9868197746697747
Recall :  0.9853333333333333


Label - 02

Removing Low-Variance Features using Variance Threshold Approach

In [18]:
variance_threshold = VarianceThreshold(threshold=0)
variance_threshold.fit(X_train['label_2'])

X_train['label_2'].columns[variance_threshold.get_support()]

low_variance_columns = []

for column in X_train['label_2'].columns:
    if (column not in X_train['label_2'].columns[variance_threshold.get_support()]):
        low_variance_columns.append(column)

print("Number of constant columns : ", len(low_variance_columns))

Number of constant columns :  0


Finding Pairwise Correlation between Features

In [19]:
def correlation(data, threshold):
    column_correlation = set()  # Set of all the names of correlated columns
    correlation_matrix = data.corr()

    for i in range(len(correlation_matrix.columns)):
        for j in range(i):
            if abs(correlation_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                column_name = correlation_matrix.columns[i]  # getting the name of column
                column_correlation.add(column_name)

    return column_correlation


correlation_features = correlation(X_train['label_2'], 0.85)
print("Number of correlation features : ", len(set(correlation_features)))
correlation_features

Number of correlation features :  0


set()

Calculate the Accuracy before applying any Feature Selection Approaches

In [20]:
xgb_regressor = xg.XGBRegressor(objective = 'reg:linear', n_estimators = 10, seed = 123)
xgb_regressor.fit(X_train['label_2'],y_train['label_2'])

y_predict = xgb_regressor.predict(X_valid['label_2']) # Predict the model

ma_error = mean_absolute_error(y_valid['label_2'], y_predict)
ms_error = mean_squared_error(y_valid['label_2'], y_predict)
r2_sc = r2_score(y_valid['label_2'], y_predict)

print("Mean Absolute Error : ", ma_error)
print("Mean Squared Error : ", ms_error)
print("R-squared (R2) Score : ", r2_sc)

[09:12:48] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
Mean Absolute Error :  2.623029861761176
Mean Squared Error :  16.77146837923947
R-squared (R2) Score :  0.6047551636813513


Univariate Feature Selection using SelectKBest Approach

In [21]:
selector = SelectKBest(score_func = f_regression, k = 180)
X_train_new_values_L2 = selector.fit_transform(X_train['label_2'],y_train['label_2'])

select_indices = np.where(selector.get_support())[0] # Get the indices of the selected features
select_feature_name = X_train['label_2'].columns[select_indices]

print("Selected Feature Names : ", select_feature_name)
print("Feature Scores : ", selector.scores_)

Selected Feature Names :  Index(['feature_1', 'feature_2', 'feature_3', 'feature_5', 'feature_6',
       'feature_7', 'feature_8', 'feature_9', 'feature_11', 'feature_12',
       ...
       'feature_236', 'feature_238', 'feature_239', 'feature_242',
       'feature_244', 'feature_248', 'feature_249', 'feature_250',
       'feature_253', 'feature_254'],
      dtype='object', length=180)
Feature Scores :  [8.43696885e+01 1.98998641e+02 4.27273993e+02 7.03816136e-01
 1.33413122e+02 2.27425407e+02 2.41111879e+02 2.31285055e+02
 2.45279579e+02 3.03920891e+00 5.82024329e+01 2.80019624e+01
 1.77957780e+01 1.26877546e+01 1.98264384e+02 1.83830683e+02
 2.56035211e+01 1.45624960e+02 5.42212226e+01 4.44120695e+01
 5.70739212e+01 2.02131846e+01 7.37512680e+01 1.89291406e+02
 1.67970646e+01 7.42718265e+00 6.42918188e+00 3.48934944e+01
 2.57597874e+02 2.63403947e+00 1.19422475e+02 8.51067603e+00
 8.80541457e+01 3.65608539e+02 6.51329640e+01 4.23394141e+02
 4.09559109e+02 2.33308107e+02 2.36662452e+0

In [22]:
xgb_regressor.fit(X_train_new_values_L2,y_train['label_2'])
y_predict = xgb_regressor.predict(selector.transform(X_valid['label_2']))

ma_error = mean_absolute_error(y_valid['label_2'], y_predict)
ms_error = mean_squared_error(y_valid['label_2'], y_predict)
r2_sc = r2_score(y_valid['label_2'], y_predict)

print("Mean Absolute Error : ", ma_error)
print("Mean Squared Error : ", ms_error)
print("R-squared (R2) Score : ", r2_sc)

[09:13:06] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
Mean Absolute Error :  2.6611624904300855
Mean Squared Error :  18.07967351180592
R-squared (R2) Score :  0.5739253453374495


Dimensionality Reduction using Principal Component Analysis Approach

In [23]:
principal_component_analysis = PCA(n_components = 0.94,svd_solver = 'full')
principal_component_analysis.fit(X_train['label_2'])
X_train_trf=pd.DataFrame(principal_component_analysis.transform(X_train['label_2']))
X_valid_trf=pd.DataFrame(principal_component_analysis.transform(X_valid['label_2']))
print('Shape after Principal Component Analysis : ',X_train_trf.shape)

xgb_regressor.fit(X_train_trf,y_train['label_2'])
y_predict = xgb_regressor.predict(principal_component_analysis.transform(X_valid['label_2']))

ma_error = mean_absolute_error(y_valid['label_2'], y_predict)
ms_error = mean_squared_error(y_valid['label_2'], y_predict)
r2_sc = r2_score(y_valid['label_2'], y_predict)

print("Mean Absolute Error : ", ma_error)
print("Mean Squared Error : ", ms_error)
print("R-squared (R2) Score : ", r2_sc)

Shape after Principal Component Analysis :  (28040, 63)
[09:13:21] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
Mean Absolute Error :  2.6495855010074116
Mean Squared Error :  18.115282050077468
R-squared (R2) Score :  0.5730861766634587


Label - 03

Removing Low-Variance Features using Variance Threshold Approach

In [24]:
variance_threshold = VarianceThreshold(threshold=0)
variance_threshold.fit(X_train['label_3'])

X_train['label_3'].columns[variance_threshold.get_support()]

low_variance_columns = []

for column in X_train['label_3'].columns:
    if (column not in X_train['label_3'].columns[variance_threshold.get_support()]):
        low_variance_columns.append(column)

print("Number of constant columns : ", len(low_variance_columns))

Number of constant columns :  0


Finding Pairwise Correlation between Features

In [25]:
def correlation(data, threshold):
    column_correlation = set()  # Set of all the names of correlated columns
    correlation_matrix = data.corr()

    for i in range(len(correlation_matrix.columns)):
        for j in range(i):
            if abs(correlation_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                column_name = correlation_matrix.columns[i]  # getting the name of column
                column_correlation.add(column_name)

    return column_correlation


correlation_features = correlation(X_train['label_3'], 0.85)
print("Number of correlation features : ", len(set(correlation_features)))
correlation_features

Number of correlation features :  0


set()

Calculate the Accuracy before applying any Feature Selection Approaches

In [26]:
classifier = RandomForestClassifier(random_state=0)
classifier.fit(X_train['label_3'],y_train['label_3'])

y_predict = classifier.predict(X_valid['label_3'])
print("Accuracy : ", metrics.accuracy_score(y_valid['label_3'],y_predict))
print("Precision : ", metrics.precision_score(y_valid['label_3'],y_predict))
print("Recall : ", metrics.recall_score(y_valid['label_3'],y_predict))

Accuracy :  0.996
Precision :  0.9967159277504105
Recall :  0.9983552631578947


Univariate Feature Selection using SelectKBest Approach

In [27]:
selector = SelectKBest(f_classif, k=190)
X_train_new_values_L3 = selector.fit_transform(X_train['label_3'], y_train['label_3'])
classifier.fit(X_train_new_values_L3, y_train['label_3'])

y_predict = classifier.predict(selector.transform(X_valid['label_3']))
print("Accuracy : ", metrics.accuracy_score(y_valid['label_3'],y_predict))
print("Precision : ", metrics.precision_score(y_valid['label_3'],y_predict))
print("Recall : ", metrics.recall_score(y_valid['label_3'],y_predict))

Accuracy :  0.9973333333333333
Precision :  0.9983552631578947
Recall :  0.9983552631578947


Dimensionality Reduction using Principal Component Analysis Approach

In [28]:
principal_component_analysis = PCA(n_components=0.93,svd_solver='full')
principal_component_analysis.fit(X_train['label_3'])

X_train_trf=pd.DataFrame(principal_component_analysis.transform(X_train['label_3']))
X_valid_trf=pd.DataFrame(principal_component_analysis.transform(X_valid['label_3']))
print('Shape after Principal Component Analysis : ', X_train_trf.shape)

classifier.fit(X_train_trf,y_train['label_3'])
y_predict = classifier.predict(principal_component_analysis.transform(X_valid['label_3']))

print("Accuracy : ", metrics.accuracy_score(y_valid['label_3'],y_predict))
print("Precision : ", metrics.precision_score(y_valid['label_3'],y_predict))
print("Recall : ", metrics.recall_score(y_valid['label_3'],y_predict))

Shape after Principal Component Analysis :  (28520, 59)
Accuracy :  0.9986666666666667
Precision :  0.9983579638752053
Recall :  1.0


Label - 04

Removing Low-Variance Features using Variance Threshold Approach

In [29]:
variance_threshold = VarianceThreshold(threshold=0)
variance_threshold.fit(X_train['label_4'])

X_train['label_4'].columns[variance_threshold.get_support()]

low_variance_columns = []

for column in X_train['label_4'].columns:
    if (column not in X_train['label_4'].columns[variance_threshold.get_support()]):
        low_variance_columns.append(column)

print("Number of constant columns : ", len(low_variance_columns))

Number of constant columns :  0


Finding Pairwise Correlation between Features

In [30]:
def correlation(data, threshold):
    column_correlation = set()  # Set of all the names of correlated columns
    correlation_matrix = data.corr()

    for i in range(len(correlation_matrix.columns)):
        for j in range(i):
            if abs(correlation_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                column_name = correlation_matrix.columns[i]  # getting the name of column
                column_correlation.add(column_name)

    return column_correlation


correlation_features = correlation(X_train['label_4'], 0.85)
print("Number of correlation features : ", len(set(correlation_features)))
correlation_features

Number of correlation features :  0


set()

Calculate the Accuracy before applying any Feature Selection Approaches

In [31]:
classifier=SVC(kernel='linear',class_weight='balanced')
classifier.fit(X_train['label_4'],y_train['label_4'])

y_predict =classifier.predict(X_valid['label_4'])

print("Accuracy : ", metrics.accuracy_score(y_valid['label_4'],y_predict))
print("Precision : ", metrics.precision_score(y_valid['label_4'],y_predict, average='weighted'))
print("Recall : ", metrics.recall_score(y_valid['label_4'],y_predict, average='weighted'))

Accuracy :  0.9373333333333334
Precision :  0.9493289124787055
Recall :  0.9373333333333334


Univariate Feature Selection using SelectKBest Approach

In [32]:
selector = SelectKBest(f_classif, k=170)
X_train_new_values_L4 = selector.fit_transform(X_train['label_4'], y_train['label_4'])
print('Shape after SelectKBest Approach : ', X_train_new_values_L4.shape)

classifier.fit(X_train_new_values_L4,y_train['label_4'])
y_predict = classifier.predict(selector.transform(X_valid['label_4']))

print("Accuracy : ", metrics.accuracy_score(y_valid['label_4'],y_predict))
print("Precision : ", metrics.precision_score(y_valid['label_4'],y_predict, average='weighted'))
print("Recall : ", metrics.recall_score(y_valid['label_4'],y_predict, average='weighted'))

Shape after SelectKBest Approach :  (28520, 170)
Accuracy :  0.9173333333333333
Precision :  0.9376409273736702
Recall :  0.9173333333333333


Dimensionality Reduction using Principal Component Analysis Approach

In [33]:
principal_component_analysis = PCA(n_components=0.98,svd_solver='full')
principal_component_analysis.fit(X_train['label_4'])

X_train_trf = pd.DataFrame(principal_component_analysis.transform(X_train['label_4']))
X_valid_trf = pd.DataFrame(principal_component_analysis.transform(X_valid['label_4']))
print('Shape after Principal Component Analysis : ', X_train_trf.shape)

classifier.fit(X_train_trf,y_train['label_4'])
y_predict = classifier.predict(principal_component_analysis.transform(X_valid['label_4']))

print("Accuracy : ", metrics.accuracy_score(y_valid['label_4'],y_predict))
print("Precision : ", metrics.precision_score(y_valid['label_4'],y_predict, average='weighted'))
print("Recall : ", metrics.recall_score(y_valid['label_4'],y_predict, average='weighted'))

Shape after Principal Component Analysis :  (28520, 89)
Accuracy :  0.848
Precision :  0.9003426427231307
Recall :  0.848
